### N-gram language models or how to write scientific papers (4 pts)

We shall train our language model on a corpora of [ArXiv](http://arxiv.org/) articles and see if we can generate a new one!

![img](https://media.npr.org/assets/img/2013/12/10/istock-18586699-monkey-computer_brick-16e5064d3378a14e0e4c2da08857efe03c04695e-s800-c85.jpg)

_data by neelshah18 from [here](https://www.kaggle.com/neelshah18/arxivdataset/)_

_Disclaimer: this has nothing to do with actual science. But it's fun, so who cares?!_

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Alternative manual download link: https://yadi.sk/d/_nGyU2IajjR9-w
!wget "https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1" -O arxivData.json.tar.gz
!tar -xvzf arxivData.json.tar.gz
data = pd.read_json("./arxivData.json")
data.sample(n=5)

--2025-10-30 13:30:32--  https://www.dropbox.com/s/99az9n1b57qkd9j/arxivData.json.tar.gz?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:6018:18::a27d:312
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1 [following]
--2025-10-30 13:30:32--  https://www.dropbox.com/scl/fi/0mulrothty5o8i8ud9gz2/arxivData.json.tar.gz?rlkey=n759u5qx2xpxxglmrl390vwvk&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucc8c2df9517d56982d0e34dd806.dl.dropboxusercontent.com/cd/0/inline/C0Pxoyn-58JMcAP_4ziWfgxWNBGSER2znLqtlNW-yyaRVu0IlU_PWvqT0EJCC8UuU7T-Hw83X_AkV7gHDzsuIxroyMsPQy4xBI1-xE-4LDR6606sD3xI6wKfryPhwRda9Rc/file?dl=1# [following]
--2025-10-30 13:30:32--  https://ucc8c2df9517d56982d0e34dd806.dl.dropboxuse

,author,day,id,link,month,summary,tag,title,year
7675,[{'name': 'Arjun Karuvally'}],21,1802.07401v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",2,One popular generative model that has high-qua...,"[{'term': 'cs.LG', 'scheme': 'http://arxiv.org...",A Study into the similarity in generator and d...,2018
1180,"[{'name': 'Arjun Jain'}, {'name': 'Jonathan To...",28,1409.7963v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",9,"In this work, we propose a novel and efficient...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",MoDeep: A Deep Learning Framework Using Motion...,2014
20436,"[{'name': 'Chen Fu'}, {'name': 'Nevin L. Zhang...",26,1601.06923v2,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",1,Objective: To treat patients with vascular mil...,"[{'term': 'cs.AI', 'scheme': 'http://arxiv.org...",Identification and classification of TCM syndr...,2016
36238,"[{'name': 'S. Mahmoud Zadeh'}, {'name': 'D. M....",9,1604.02523v4,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,The AUV three-dimension path planning in compl...,"[{'term': 'cs.RO', 'scheme': 'http://arxiv.org...",Differential Evolution for Efficient AUV Path ...,2016
22674,[{'name': 'Fionn Murtagh'}],6,1704.01871v1,"[{'rel': 'alternate', 'href': 'http://arxiv.or...",4,Cluster analysis of very high dimensional data...,"[{'term': 'stat.ML', 'scheme': 'http://arxiv.o...",Massive Data Clustering in Moderate Dimensions...,2017


In [3]:
# assemble lines: concatenate title and description
lines = data.apply(lambda row: row['title'] + ' ; ' + row['summary'].replace("\n", ' '), axis=1).tolist()

sorted(lines, key=len)[:3]

['Differential Contrastive Divergence ; This paper has been retracted.',
 'What Does Artificial Life Tell Us About Death? ; Short philosophical essay',
 'P=NP ; We claim to resolve the P=?NP problem via a formal argument for P=NP.']

### Tokenization

You know the dril. The data is messy. Go clean the data. Use WordPunctTokenizer or something.


In [5]:
# Task: convert lines (in-place) into strings of space-separated tokens. Import & use WordPunctTokenizer
from nltk.tokenize import WordPunctTokenizer
tokenizer = WordPunctTokenizer()
lines = [' '.join(tokenizer.tokenize(line.lower())) for line in lines]
print(sorted(lines, key=len)[:3])

['differential contrastive divergence ; this paper has been retracted .', 'what does artificial life tell us about death ? ; short philosophical essay', 'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .']


In [6]:
assert sorted(lines, key=len)[0] == \
    'differential contrastive divergence ; this paper has been retracted .'
assert sorted(lines, key=len)[2] == \
    'p = np ; we claim to resolve the p =? np problem via a formal argument for p = np .'

### N-Gram Language Model (1point)

A language model is a probabilistic model that estimates text probability: the joint probability of all tokens $w_t$ in text $X$: $P(X) = P(w_1, \dots, w_T)$.

It can do so by following the chain rule:
$$P(w_1, \dots, w_T) = P(w_1)P(w_2 \mid w_1)\dots P(w_T \mid w_1, \dots, w_{T-1}).$$

The problem with such approach is that the final term $P(w_T \mid w_1, \dots, w_{T-1})$ depends on $n-1$ previous words. This probability is impractical to estimate for long texts, e.g. $T = 1000$.

One popular approximation is to assume that next word only depends on a finite amount of previous words:

$$P(w_t \mid w_1, \dots, w_{t - 1}) = P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1})$$

Such model is called __n-gram language model__ where n is a parameter. For example, in 3-gram language model, each word only depends on 2 previous words.

$$
    P(w_1, \dots, w_n) = \prod_t P(w_t \mid w_{t - n + 1}, \dots, w_{t - 1}).
$$

You can also sometimes see such approximation under the name of _n-th order markov assumption_.

The first stage to building such a model is counting all word occurences given N-1 previous words

In [8]:
from tqdm import tqdm
from collections import defaultdict, Counter

UNK, EOS = "_UNK_", "_EOS_"

def count_ngrams(lines, n):
    counts = defaultdict(Counter)

    for line in tqdm(lines):
        tokens = line.split()
        tokens.append(EOS)
        for i in range(len(tokens)):
            start = i - (n - 1)
            prefix = tokens[start:i] if start >= 0 else [UNK] * (-start) + tokens[0:i]
            prefix = tuple(prefix[-(n - 1):]) if n > 1 else tuple()
            next_token = tokens[i]
            counts[prefix][next_token] += 1

    return counts

dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)

assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts[('_UNK_', 'a')]['note'] == 3
assert dummy_counts[('p', '=')]['np'] == 2
assert dummy_counts[('author', '.')][EOS] == 1
print("All tests passed!")



100%|██████████| 100/100 [00:00<00:00, 15301.54it/s]

All tests passed!


In [9]:
# let's test it
dummy_lines = sorted(lines, key=len)[:100]
dummy_counts = count_ngrams(dummy_lines, n=3)
assert set(map(len, dummy_counts.keys())) == {2}, "please only count {n-1}-grams"
assert len(dummy_counts[('_UNK_', '_UNK_')]) == 78
assert dummy_counts['_UNK_', 'a']['note'] == 3
assert dummy_counts['p', '=']['np'] == 2
assert dummy_counts['author', '.']['_EOS_'] == 1

100%|██████████| 100/100 [00:00<00:00, 17383.55it/s]


Once we can count N-grams, we can build a probabilistic language model.
The simplest way to compute probabilities is in proporiton to counts:

$$ P(w_t | prefix) = { Count(prefix, w_t) \over \sum_{\hat w} Count(prefix, \hat w) } $$

In [10]:
from collections import defaultdict, Counter

class NGramLanguageModel:
    def __init__(self, lines, n):
        """
        Train a simple count-based language model:
        compute probabilities P(w_t | prefix) given ngram counts
        """
        assert n >= 1
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.probs = defaultdict(Counter)
        for prefix, next_tokens in counts.items():
            total = sum(next_tokens.values())
            for token, cnt in next_tokens.items():
                self.probs[prefix][token] = cnt / total

    def get_possible_next_tokens(self, prefix):
        """
        :param prefix: string with space-separated prefix tokens
        :returns: a dictionary {token : probability}
        """
        prefix = prefix.split()
        prefix = prefix[max(0, len(prefix) - self.n + 1):]
        prefix = [UNK] * (self.n - 1 - len(prefix)) + prefix
        return self.probs[tuple(prefix)]

    def get_next_token_prob(self, prefix, next_token):
        """
        Returns P(next_token | prefix)
        """
        return self.get_possible_next_tokens(prefix).get(next_token, 0)


Let's test it!

In [11]:
dummy_lm = NGramLanguageModel(dummy_lines, n=3)

p_initial = dummy_lm.get_possible_next_tokens('') # '' -> ['_UNK_', '_UNK_']
assert np.allclose(p_initial['learning'], 0.02)
assert np.allclose(p_initial['a'], 0.13)
assert np.allclose(p_initial.get('meow', 0), 0)
assert np.allclose(sum(p_initial.values()), 1)

p_a = dummy_lm.get_possible_next_tokens('a') # '' -> ['_UNK_', 'a']
assert np.allclose(p_a['machine'], 0.15384615)
assert np.allclose(p_a['note'], 0.23076923)
assert np.allclose(p_a.get('the', 0), 0)
assert np.allclose(sum(p_a.values()), 1)

assert np.allclose(dummy_lm.get_possible_next_tokens('a note')['on'], 1)
assert dummy_lm.get_possible_next_tokens('a machine') == \
    dummy_lm.get_possible_next_tokens("there have always been ghosts in a machine"), \
    "your 3-gram model should only depend on 2 previous words"

100%|██████████| 100/100 [00:00<00:00, 10682.04it/s]


Now that you've got a working n-gram language model, let's see what sequences it can generate. But first, let's train it on the whole dataset.

In [12]:
lm = NGramLanguageModel(lines, n=3)

100%|██████████| 41000/41000 [00:18<00:00, 2267.51it/s]


The process of generating sequences is... well, it's sequential. You maintain a list of tokens and iteratively add next token by sampling with probabilities.

$ X = [] $

__forever:__
* $w_{next} \sim P(w_{next} | X)$
* $X = concat(X, w_{next})$


Instead of sampling with probabilities, one can also try always taking most likely token, sampling among top-K most likely tokens or sampling with temperature. In the latter case (temperature), one samples from

$$w_{next} \sim {P(w_{next} | X) ^ {1 / \tau} \over \sum_{\hat w} P(\hat w | X) ^ {1 / \tau}}$$

Where $\tau > 0$ is model temperature. If $\tau << 1$, more likely tokens will be sampled with even higher probability while less likely tokens will vanish.

In [13]:
import numpy as np

def get_next_token(lm, prefix, temperature=1.0):
    """
    return next token after prefix;
    :param temperature: samples proportionally to lm probabilities ^ (1 / temperature)
        if temperature == 0, always takes most likely token. Break ties arbitrarily.
    """
    probs_dict = lm.get_possible_next_tokens(prefix)
    if not probs_dict:
        return EOS

    tokens = np.array(list(probs_dict.keys()))
    probs = np.array(list(probs_dict.values()), dtype=np.float64)
    if temperature == 0:
        return tokens[np.argmax(probs)]
    adjusted_probs = probs ** (1.0 / temperature)
    adjusted_probs = adjusted_probs / adjusted_probs.sum()
    next_token = np.random.choice(tokens, p=adjusted_probs)
    return next_token


In [14]:
from collections import Counter
test_freqs = Counter([get_next_token(lm, 'there have') for _ in range(10000)])
assert 250 < test_freqs['not'] < 450
assert 8500 < test_freqs['been'] < 9500
assert 1 < test_freqs['lately'] < 200

test_freqs = Counter([get_next_token(lm, 'deep', temperature=1.0) for _ in range(10000)])
assert 1500 < test_freqs['learning'] < 3000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.5) for _ in range(10000)])
assert 8000 < test_freqs['learning'] < 9000
test_freqs = Counter([get_next_token(lm, 'deep', temperature=0.0) for _ in range(10000)])
assert test_freqs['learning'] == 10000

print("Looks nice!")

Looks nice!


Let's have fun with this model

In [15]:
prefix = 'artificial' # <- your ideas :)

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

artificial learning in dialogue , request dialogue , accurate , while other distributed multi - task learning to compose the images . an urgent requirement to determine the correct viewpoint of wqos to the signal . a . k - medoids partition the vertices in dendrograms are given a network of cameras with stereo images , in image and the ability of the mammalian brain appears alongside other layers . these networks is very effective way of their objects . they produce a hidden markov models . this includes the two sorted clp language , profanity or abusive statements . conventional


In [16]:
prefix = 'bridging the' # <- more of your ideas

for i in range(100):
    prefix += ' ' + get_next_token(lm, prefix, temperature=0.5)
    if prefix.endswith(EOS) or len(lm.get_possible_next_tokens(prefix)) == 0:
        break

print(prefix)

bridging the gap between the two - dimensional data . the results of the network . we propose a new approach to the best of our approach is based on the complexity of the new model , which we call this technique , we apply a state - of - the - art . _EOS_


__More in the homework:__ nucleus sampling, top-k sampling, beam search(not for the faint of heart).

### Evaluating language models: perplexity (1point)

Perplexity is a measure of how well your model approximates the true probability distribution behind the data. __Smaller perplexity = better model__.

To compute perplexity on one sentence, use:
$$
    {\mathbb{P}}(w_1 \dots w_N) = P(w_1, \dots, w_N)^{-\frac1N} = \left( \prod_t P(w_t \mid w_{t - n}, \dots, w_{t - 1})\right)^{-\frac1N},
$$


On the corpora level, perplexity is a product of probabilities of all tokens in all sentences to the power of $1/N$, where $N$ is __total length (in tokens) of all sentences__ in corpora.

This number can quickly get too small for float32/float64 precision, so we recommend you to first compute log-perplexity (from log-probabilities) and then take the exponent.

In [18]:
import numpy as np

def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: list of strings (each line = space-separated tokens)
    :param min_logprob: lower bound for log-probabilities
    :returns: corpus-level perplexity
    """
    total_logprob = 0.0
    total_tokens = 0

    for line in lines:
        tokens = line.strip().split() + [EOS]
        prefix = []
        for token in tokens:
            log_p = np.log(lm.get_next_token_prob(prefix, token) + 1e-12)
            log_p = max(log_p, min_logprob)
            total_logprob += log_p
            total_tokens += 1
            prefix.append(token)
    avg_logprob = total_logprob / total_tokens
    ppl = np.exp(-avg_logprob)

    return ppl

In [21]:
import numpy as np

def perplexity(lm, lines, min_logprob=np.log(10 ** -50.)):
    """
    :param lines: list of strings (each line = space-separated tokens)
    :param min_logprob: lower bound for log-probabilities
    :returns: corpus-level perplexity
    """
    total_logprob = 0.0
    total_tokens = 0
    for line in lines:
        tokens = line.strip().split()
        prefix = []
        for token in tokens + [EOS]:
            prob = lm.get_next_token_prob(prefix, token)
            logprob = np.log(prob) if prob > 0 else min_logprob
            logprob = max(logprob, min_logprob)
            total_logprob += logprob
            total_tokens += 1
            prefix.append(token)
    avg_logprob = total_logprob / total_tokens
    ppl = np.exp(-avg_logprob)
    return ppl

Now let's measure the actual perplexity: we'll split the data into train and test and score model on test data only.

In [24]:
import numpy as np
from tqdm import tqdm
from collections import defaultdict, Counter
from sklearn.model_selection import train_test_split
UNK = "_UNK_"
EOS = "_EOS_"
def count_ngrams(lines, n, min_count=1):
    """
    Считает, сколько раз каждый токен встречается после (n-1) предыдущих.
    """
    token_counts = Counter()
    for line in lines:
        token_counts.update(line.split())
    def process_line(line):
        tokens = [t if token_counts[t] >= min_count else UNK for t in line.split()]
        tokens += [EOS]
        return tokens
    counts = defaultdict(Counter)
    for line in lines:
        tokens = process_line(line)
        for i in range(len(tokens)):
            prefix = tokens[max(0, i - n + 1):i]
            prefix = [UNK] * (n - 1 - len(prefix)) + prefix
            nex

In [25]:
# whoops, it just blew up :)

### LM Smoothing

The problem with our simple language model is that whenever it encounters an n-gram it has never seen before, it assigns it with the probabilitiy of 0. Every time this happens, perplexity explodes.

To battle this issue, there's a technique called __smoothing__. The core idea is to modify counts in a way that prevents probabilities from getting too low. The simplest algorithm here is Additive smoothing (aka [Lapace smoothing](https://en.wikipedia.org/wiki/Additive_smoothing)):

$$ P(w_t | prefix) = { Count(prefix, w_t) + \delta \over \sum_{\hat w} (Count(prefix, \hat w) + \delta) } $$

If counts for a given prefix are low, additive smoothing will adjust probabilities to a more uniform distribution. Not that the summation in the denominator goes over _all words in the vocabulary_.

Here's an example code we've implemented for you:

In [26]:
class LaplaceLanguageModel(NGramLanguageModel):
    """ this code is an example, no need to change anything """
    def __init__(self, lines, n, delta=1.0):
        self.n = n
        counts = count_ngrams(lines, self.n)
        self.vocab = set(token for token_counts in counts.values() for token in token_counts)
        self.probs = defaultdict(Counter)

        for prefix in counts:
            token_counts = counts[prefix]
            total_count = sum(token_counts.values()) + delta * len(self.vocab)
            self.probs[prefix] = {token: (token_counts[token] + delta) / total_count
                                          for token in token_counts}
    def get_possible_next_tokens(self, prefix):
        token_probs = super().get_possible_next_tokens(prefix)
        missing_prob_total = 1.0 - sum(token_probs.values())
        missing_prob = missing_prob_total / max(1, len(self.vocab) - len(token_probs))
        return {token: token_probs.get(token, missing_prob) for token in self.vocab}

    def get_next_token_prob(self, prefix, next_token):
        token_probs = super().get_possible_next_tokens(prefix)
        if next_token in token_probs:
            return token_probs[next_token]
        else:
            missing_prob_total = 1.0 - sum(token_probs.values())
            missing_prob_total = max(0, missing_prob_total) # prevent rounding errors
            return missing_prob_total / max(1, len(self.vocab) - len(token_probs))


**Disclaimer**: the implementation above assumes all words unknown within a given context to be equally likely, *as well as the words outside of vocabulary*. Therefore, its' perplexity will be lower than it should when encountering such words. Therefore, comparing it with a model with fewer unknown words will not be fair. When implementing your own smoothing, you may handle this by adding a virtual `UNK` token of non-zero probability. Technically, this will result in a model where probabilities do not add up to $1$, but it is close enough for a practice excercise.

In [39]:
for n in (1, 2, 3):
    dummy_lm = LaplaceLanguageModel(dummy_lines, n=n)
    total_prob = sum([dummy_lm.get_next_token_prob(['a'], w_i) for w_i in dummy_lm.vocab])
    print(f"N={n}, total probability sum = {total_prob:.6f}")
    assert np.allclose(total_prob, 1), "I told you not to break anything! :)"

N=1, total probability sum = 1.000000
N=2, total probability sum = 1.000000
N=3, total probability sum = 1.000000


In [41]:
for n in (1, 2, 3):
    lm = LaplaceLanguageModel(train_lines, n=n, delta=0.1)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))

N = 1, Perplexity = 53543.26887
N = 2, Perplexity = 498.98755
N = 3, Perplexity = 3779.14562


In [42]:
# optional: try to sample tokens from such a model

### Kneser-Ney smoothing (2 points)

Additive smoothing is simple, reasonably good but definitely not a State of The Art algorithm.


Your final task in this notebook is to implement [Kneser-Ney](https://en.wikipedia.org/wiki/Kneser%E2%80%93Ney_smoothing) smoothing.

It can be computed recurrently, for n>1:

$$P_{kn}(w_t | prefix_{n-1}) = { \max(0, Count(prefix_{n-1}, w_t) - \delta) \over \sum_{\hat w} Count(prefix_{n-1}, \hat w)} + \lambda_{prefix_{n-1}} \cdot P_{kn}(w_t | prefix_{n-2})$$

where
- $prefix_{n-1}$ is a tuple of {n-1} previous tokens
- $lambda_{prefix_{n-1}}$ is a normalization constant chosen so that probabilities add up to 1
- Unigram $P_{kn}(w_t | prefix_{n-2})$ corresponds to Kneser Ney smoothing for {N-1}-gram language model.
- Unigram $P_{kn}(w_t)$ is a special case: how likely it is to see x_t in an unfamiliar context

See lecture slides or wiki for more detailed formulae.

__Your task__ is to
- implement `KneserNeyLanguageModel` class,
- test it on 1-3 gram language models
- find optimal (within reason) smoothing delta for 3-gram language model with Kneser-Ney smoothing

In [43]:
from collections import defaultdict, Counter
import numpy as np

class KneserNeyLanguageModel(NGramLanguageModel):
    """ Kneser-Ney smoothing language model """
    def __init__(self, lines, n, delta=0.75):
        self.n = n
        self.delta = delta
        self.counts = count_ngrams(lines, n)
        self.lower_counts = count_ngrams(lines, n - 1)
        self.vocab = set(token for token_counts in self.counts.values() for token in token_counts)
        continuation_counts = Counter()
        for prefix in self.counts:
            for token in self.counts[prefix]:
                continuation_counts[token] += 1
        self.continuation_counts = continuation_counts
        self.total_continuations = sum(continuation_counts.values())
    def continuation_prob(self, token):
        """ Unigram Kneser-Ney probability (probability of seeing token in new context) """
        return self.continuation_counts[token] / self.total_continuations
    def get_next_token_prob(self, prefix, next_token):
        """ Recursive Kneser-Ney smoothing """
        prefix = tuple(prefix[-(self.n - 1):])
        if self.n == 1:
            return self.continuation_prob(next_token)
        count_prefix = sum(self.counts[prefix].values()) if prefix in self.counts else 0
        count_ngram = self.counts[prefix][next_token] if next_token in self.counts.get(prefix, {}) else 0
        p_cont = 0
        if count_prefix > 0:
            p_cont = max(count_ngram - self.delta, 0) / count_prefix
        unique_continuations = len(self.counts[prefix]) if prefix in self.counts else 0
        lambda_prefix = (self.delta * unique_continuations / count_prefix) if count_prefix > 0 else 0
        lower_model = KneserNeyLanguageModel([], n=self.n - 1, delta=self.delta)
        lower_model.counts = self.lower_counts
        lower_model.continuation_counts = self.continuation_counts
        lower_model.total_continuations = self.total_continuations
        return p_cont + lambda_prefix * lower_model.get_next_token_prob(prefix[1:], next_token)
    def get_possible_next_tokens(self, prefix):
        """ Возвращает распределение вероятностей всех слов """
        return {token: self.get_next_token_prob(prefix, token) for token in self.vocab}

In [44]:
#test that it's a valid probability model
for n in (1, 2, 3):
    dummy_lm = KneserNeyLanguageModel(dummy_lines, n=n)
    assert np.allclose(sum([dummy_lm.get_next_token_prob('a', w_i) for w_i in dummy_lm.vocab]), 1), "I told you not to break anything! :)"

In [46]:
for n in (1, 2, 3):
    lm = KneserNeyLanguageModel(train_lines, n=n, delta=0.75)
    ppx = perplexity(lm, test_lines)
    print("N = %i, Perplexity = %.5f" % (n, ppx))


N = 1, Perplexity = 198615.81919
N = 2, Perplexity = 1499.05130
N = 3, Perplexity = 417272762.71935
